In [ ]:
Context:

We will go through a basic Long Short Term Memory (LSTM) model for time series and investigate

First load in the data. The preproccessing only consist of normalization and the creation of windows.
Creation of the LSTM model
Training the LSTM model
Testing the LSTM model with 1 time step and with 1 window

Context:

We will go through a basic Long Short Term Memory (LSTM) model for time series and investigate the maximal efficacy of different sets of parameters

The broad overview of the steps will involve loading the data, preprocessing of the data, creation of the LSTM model, training of the model and finally testing of the model.

Task 1 - 0 points) Run the necessary libraries, which have been provided for you below

In [29]:
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
import plotly

# import the relevant Keras modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

import yfinance as yf
import plotly.graph_objects as go
from keras.callbacks import EarlyStopping

Task 2 - 10 points) Load the data

Retrieve Close price of Google from yahoo finance from 2008-01-02 to 2024-12-30.

Refer to the instructions in homework 2, if necessary, for up to date instructions on how to retrieve such data for free under new Yahoo Finance API permissions.

In [16]:
ticker = 'GOOG'
start = '2008-01-02'
end = '2024-12-30'

data = yf.download(ticker, start=start, end=end, auto_adjust=False)

adj_close = data[['Adj Close']]

[*********************100%***********************]  1 of 1 completed


In [17]:
adj_close.isna().sum().sum()

np.int64(0)

Task 3 - 10 points) Write a function to visualise the data to make sure it has been succesfully imported

In [9]:
def time_series_visualizer(data: pd.DataFrame):
    assert "Adj Close" in data.columns

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'],
                             mode='lines',
                             name='Adjusted Close'))

    fig.update_layout(title='GOOG Adjusted Close',
                      xaxis_title='Date',
                      yaxis_title='Price (USD)',
                      template='plotly_white',
                      height=600,
                      width=1000)

    fig.show()

In [20]:
data.columns = data.columns.droplevel(1)
data.head()

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2008-01-02,16.985332,17.065783,17.369146,16.879980,17.257067,172921733
2008-01-03,16.988804,17.069269,17.107128,16.849842,17.067528,130587647
2008-01-04,16.286524,16.363665,16.960428,16.313852,16.928797,215195594
2008-01-07,16.094408,16.170637,16.495173,15.874249,16.287451,257096061
2008-01-08,15.658861,15.733029,16.437389,15.716092,16.264038,214364490


In [21]:
time_series_visualizer(data)

Context) Now we will create windows of default length 20 with the data imported and begin the process of constructing the model.

Task 4.1 - 5 points) Define the parameter for the length of your window

In [22]:
window_size = 20

Task 4.2 - 5 points) Create a data point which splits training and testing set

In [24]:
split = 0.8
split_date = data.index[int(split * len(data.index))]
print(split_date)

2021-08-04 00:00:00


Task 4.3 - 5 points) Create a window for training

In [33]:
# The return prediction will be done between 1 and 10 days ahead
train_returns_h = {h: [] for h in range(1, 11)}

train_data = data.loc[:split_date, 'Adj Close']
train_windows = []

for i in range(len(train_data) - window_size - 10):
    window = train_data[i:i+window_size].values
    train_windows.append(window)

    for h in range(1, 11):
        current_day = train_data[i + window_size - 1]
        future_day = train_data[i + window_size - 1 + h]
        daily_return = (future_day - current_day) / current_day
        train_returns_h[h].append(daily_return)

<ipython-input-33-a5875e82a5f0>:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-33-a5875e82a5f0>:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Task 4.4 - 5 points) Create a window for testing

In [32]:
# Same thing happens for the testing set
test_returns_h = {h: [] for h in range(1, 11)}

test_data = data.loc[split_date:, 'Adj Close']
test_windows = []

for i in range(len(test_data) - window_size - 10):
    window = test_data[i:i+window_size].values
    test_windows.append(window)

    for h in range(1, 11):
        current_day = test_data[i + window_size - 1]
        future_day = test_data[i + window_size - 1 + h]
        daily_return = (future_day - current_day) / current_day
        test_returns_h[h].append(daily_return)

<ipython-input-32-a6ba8ed6a09a>:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-32-a6ba8ed6a09a>:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Task 5 - 10 points) Create a function in which you execute the building of your LSTM model

Consider these inputs:

Activation function +
Loss function +
Dropout rate +
Optimizer +
nn layers/architecture

In [27]:
def build_lstm(input_shape, activation = 'tanh', loss_function = 'mse', dropout_rate = 0.2, optimizer = 'adam', lstm_units = [50, 50]):
    model = Sequential()

    for i, units in enumerate(lstm_units):
        return_seq = i != (len(lstm_units) - 1)
        model.add(LSTM(units=units, activation=activation,
                       return_sequences=return_seq,
                       input_shape=input_shape if i == 0 else None))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1))

    model.compile(loss=loss_function, optimizer=optimizer)
    return model

In [28]:
input_shape = (window_size, 1)
test = build_lstm(input_shape,
                   activation='tanh',
                   loss_function='mse',
                   dropout_rate=0.3,
                   optimizer='adam',
                   lstm_units=[50, 25])

test.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 25)             │         7,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,026 (70.41 KB)

 Trainable params: 18,026 (70.41 KB)

 Non-trainable params: 0 (0.00 B)

Task 6 - 10 points) Considering epochs as a parameter to adjust, use epochs to set the stopping condition of the model by monitoring the loss at each step of the iteration.

In [34]:
def early_stopping(monitor = 'loss', patience = 5, restore_best_weights = True):
    return EarlyStopping(monitor=monitor,
                         patience=patience,
                         restore_best_weights=restore_best_weights)

Task 7 - 10 points) Plot the performance of the model, in terms of accuracy, when predicting one time point ahead compared to multiple time points ahead.

Task 8 - 10 points) Test the work attempted with different set of parameters.
Specifically test the functions by attempting to modify the following parameters and report, for each, your findings on their impact within the model.

1) Window length
2) LSTM parameter: activation function
3) LSTM parameter: loss function
4) LSTM parameter: dropout rate
5) LSTM parameter: optimizer

Task 9 - 20 points):

In this short NLP task, you will use an LSTM model to perform next-word prediction on a small text sample.

Instructions:
	1.	Select a short paragraph of English text (around 100–200 words).
	2.	Tokenize the text into words, and create input-output pairs where each input consists of 3 consecutive words and the output is the 4th word.
	3.	Convert words to integer indices (you may use Keras’ Tokenizer).
	4.	Train a simple LSTM model to predict the next word given the previous 3.
	5.	Test your model by providing a custom 3-word input and printing the predicted next word.

You do not need to optimize the model. Just focus on building a viable version of the workflow.